# Механизмы внимания и трансформеры

Одним из основных недостатков рекуррентных сетей является то, что все слова в последовательности оказывают одинаковое влияние на результат. Кроме того, для задач sequence-to-sequence, таких как распознавание именованных сущностей и машинный перевод, нам необходимо запомнить всю входную последовательность в единый вектор смысла. В действительности конкретные слова во входной последовательности часто оказывают большее влияние на определённые выходы, чем другие.

Рассмотрим задачу sequence-to-sequence, такую, как машинный перевод. Она реализуется двумя рекуррентными сетями, где одна сеть (**кодировщик**) сворачивает входную последовательность в скрытое состояние, а другая, **декодер**, разворачивает это скрытое состояние в результирующую выходную последовательность. Проблема с этим подходом заключается в том, что конечному состоянию сети будет трудно запомнить начало предложения, что приведет к низкому качеству модели на длинных предложениях.

**Механизмы внимания** позволяют бороться с этой проблемой. Они представляют собой средство взвешивания контекстуального воздействия каждого входного токена на каждое выходное предсказание RNN. Внимание реализуется путем создания связей между промежуточными состояниями энкодера и декодера. Таким образом, при генерации выходного символа $y_t$, мы будем учитывать все входные скрытые состояния $h_i$, с различными весовыми коэффициентами $\alpha_{t,i}$. 

![Image showing an encoder/decoder model with an additive attention layer](images/encoder-decoder-attention.png)

(Тут показана модель энкодера-декодера с аддитивным механизмом внимания из [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), цитируемая из [этого сообщения в блоге](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html))

Матрица внимания $\{\alpha_{i,j}\}$ будет определять степень, в которой те или иные входные слова влияют нагенерацию данного слова в выходной последовательности. Ниже приведен пример такой матрицы:

![Image showing a sample alignment found by RNNsearch-50, taken from Bahdanau - arviz.org](images/bahdanau-fig3.png)

(Рисунок взят из [Богданау и др., 2015](https://arxiv.org/pdf/1409.0473.pdf) (рис.3))

Все современные подходы к NLP используют механизмы внимания. Однако добавление внимания значительно увеличивает количество параметров модели, что приводит к проблемам масштабирования. Ключевым ограничением масштабирования RNN является то, что рекуррентный характер моделей затрудняет параллельное обучение моделей на кластерах GPU, в то время как размер модели становится настолько большим, что обучать её на одном процессоре становится слишком долго. В RNN каждый элемент последовательности должен быть обработан в последовательном порядке, что означает, что процесс обучения не может быть легко распараллелен.

Внедрение механизмов внимания и преодоление такого ограничения позволило обучать действительно большие модели, такие, как BERT или GPT-3.

## Трансформеры

В отличие от рекуррентных сетей, которые последовательно передают контекст из одного блока в другой, **модели трансформеров** используют **позиционную кодировку** и **внимание**, одновременно рассматривая всё предложение и вычисляя коэффициент внимания между словами входного предложения, и между входным и выходным предложением.

![Animated GIF showing how the evaluations are performed in transformer models.](images/transformer-animated-explanation.gif) 

Такой процесс, в отличие от последовательного применения рекуррентной сети, может распараллеливаться лучше, что позволяет создавать гораздо более крупные и выразительные языковые модели. 

Подробнее про трансформерные модели можно почитать [в этой русскоязычной статье](https://sysblok.ru/knowhow/kak-rabotajut-transformery-krutejshie-nejroseti-nashih-dnej/) (или [здесь](http://jalammar.github.io/illustrated-transformer/) на английском языке).

Важной идеей трансформерных моделей является идея **внутреннего внимания** (*self-attention*). Например, рассмотрим два предложения:

* Chicken did not cross the road because it was too wide
* Chicken did not cross the road because it was too tired

В первом случае слово *it* относится к дороге, а во втором - к цыплёнку. Энкодер предложения должен учитывать это обстоятельство, и механизм внутреннего внимания как раз помогает построить матрицу взаимного влияния токенов друг на друга в рамках входной последовательности.

## Построение простой модели трансформера

Keras не содержит встроенного слоя для трансформеров, но мы можем построить свой собственный. Такой процесс неплохо описан [в документации](https://www.tensorflow.org/text/tutorials/transformer).

Как и прежде, мы сосредоточимся на текстовой классификации набора данных AG News, но стоит отметить, что трансформерные модели показывают наилучший результат и в существенно более сложных задачах NLP.

In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

Metal device set to: Apple M2

systemMemory: 24.00 GB
maxCacheSize: 8.00 GB



2022-12-04 11:24:11.221686: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-04 11:24:11.221804: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Новые слои в Keras должны наследовать класс `Layer` и реализовывать метод `call`. Начнем с слоя **позиционного эмбеддинга**, который применяется ко входной последовательности. Мы будем использовать [код из официальной документации Keras](https://keras.io/examples/nlp/text_classification_with_transformer/). Также предположим, что все входные последовательности приведены к длине `maxlen`.

In [4]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

Этот слой состоит из двух слоев `Embedding`: для кодирования токенов (способом, который мы обсуждали ранее) и позиций токенов. Позиции токенов создаются в виде последовательности натуральных чисел от 0 до `maxlen` с помощью `tf.range`, а затем передаются через слой эмбеддинга. Два результирующих эмбеддинг-вектора складываются, что приводит к тензору размерности `maxlen`$\times$`embed_dim`.

<img src="images/pos-embedding.png" width="40%"/>

Теперь давайте реализуем трансформерный блок. В качестве входа он воспринимает выход предыдущего блока позиционного эмбеддинга:

In [5]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

Мы используем блок `MultiHeadAttention` для получения вектора внутреннего внимания (*self-attention*) размерности `maxlen`$\times$`embed_dim` - для этого на вход слою мы подаём одну и ту же последовательность `inputs`. Мы используем приём, аналогичный residual connection в сетях ResNet для стабилизации обучения, смешивая результат внутреннего внимания с входом, и нормализуя его с помощью `LayerNormalizaton`.

> **Примечание**: `LayerNormalization` похож на `BatchNormalization`, но он нормализует выходы предыдущего слоя для каждого обучающего образца независимо друг от друга, чтобы привести их к диапазону [-1..1].

Выход этого слоя затем пропускается через полносвязную сеть (в нашем случае — двухслойный персептрон), и результат добавляется к конечному выходу (который снова проходит нормализацию).

<img src="images/transformer-layer.png" width="30%" />

Теперь мы можем определить полную модель трансформера:

In [6]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_1 (TextV  (None, 256)              0         
 ectorization)                                                   
                                                                 
 token_and_position_embeddin  (None, 256, 32)          648192    
 g_1 (TokenAndPositionEmbedd                                     
 ing)                                                            
                                                                 
 transformer_block (Transfor  (None, 256, 32)          10656     
 merBlock)                                                       
                                                                 
 global_average_pooling1d (G  (None, 32)               0         
 lobalAveragePooling1D)                                          
                                                      

In [7]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer


2022-12-04 11:45:20.282332: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-12-04 11:45:20.372871: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


NotFoundError: Graph execution error:

Detected at node 'StringSplit/stack' defined at (most recent call last):
    File "/opt/conda/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/opt/conda/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/opt/conda/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/opt/conda/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/opt/conda/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/opt/conda/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/opt/conda/lib/python3.9/asyncio/base_events.py", line 601, in run_forever
      self._run_once()
    File "/opt/conda/lib/python3.9/asyncio

## BERT

**BERT** (Bidirectional Encoder Representations from Transformers) is a very large multi layer transformer network with 12 layers for *BERT-base*, and 24 for *BERT-large*. The model is first pre-trained on large corpus of text data (WikiPedia + books) using unsupervised training (predicting masked words in a sentence). During pre-training the model absorbs significant level of language understanding which can then be leveraged with other datasets using fine tuning. This process is called **transfer learning**. 


**BERT** (Bidirectional Encoder Representations from Transformers) представляет собой большую многослойную трансформерную сеть с 12 слоями для *BERT-base* и 24 - для *BERT-large*. Модель сначала предварительно обучается на большом корпусе текстовых данных (Википедия + книги) с использованием self-supervised learning (прогнозирование замаскированных слов в предложении). Во время предварительного обучения модель приобретает значительный уровень понимания языка в целом, который затем может быть использован с другими наборами данных для решения специфических задач. Этот процесс называется **трансферное обучение**, и хорошо описан [в официальной документации Keras](https://keras.io/examples/nlp/masked_language_modeling/).

![picture from http://jalammar.github.io/illustrated-bert/](./images/jalammarBERT-language-modeling-masked-lm.png)

Существует множество вариаций предобученных трансформерных архитектур, которые можно использовать для трансферного обучения, включая BERT, DistilBERT, BigBird, OpenGPT3 и другие. 

Давайте посмотрим, как мы можем использовать предварительно обученную модель BERT для решения нашей традиционной задачи классификации новостей. Мы позаимствуем идею и некоторый код из [официальной документации](https://www.tensorflow.org/text/tutorials/classify_text_with_bert).

Для загрузки предварительно обученных моделей мы будем использовать **Tensorflow hub**. Загрузим векторизатор, специально обученный  для BERT:

In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

Важно использовать тот же векторизатор, на котором была обучена исходная сеть. Векторизатор BERT возвращает три компонента:
* `input_word_ids` - последовательность номеров токенов для входного предложения
* `input_mask` показывает, какая часть последовательности содержит фактические входные данные, а какая - padding. Он похож на маску, создаваемую слоем `Masking`
* `input_type_ids` используется для более сложных задач языкового моделирования (например, ответы на вопросы), и позволяет указать два входных предложения в одной последовательности.

Затем мы можем создать экземпляр BERT:

In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


Смотрим на то, что возвращает сеть BERT:
* `pooled_output` является результатом усреднения всех веторов последнего слоя. Вы можете рассматривать его как семантический вектор для всей входной последовательности. Он эквивалентен выходу слоя `GlobalAveragePooling1D` в нашей предыдущей модели.
* `sequence_output` - это выход последнего слоя трансформера (соответствует выходу `TransformerBlock` в нашей модели выше)
* `encoder_outputs` - это выходы всех промежуточных слоёв сети. Поскольку мы загрузили 4-слойную модель BERT (как вы, вероятно, можете догадаться из названия, которое содержит «4_H»), этот выход содержит 4 тензора. Последний из них совпадает с `sequence_output`.

Теперь определим сквозную модель классификации. Мы будем использовать *функциональное определение модели* в Keras. Мы также сделаем веса модели BERT необучаемыми и обучим только окончательный классификатор:

In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


Несмотря на то, что обучаемых параметров немного, процесс довольно медленный, потому что экстрактор признаков BERT является весьма сложной моделью. На одной эпохе обучения точность не слишком высока, можно попробовать продолжить обучение, или использовать несколько других приёмов.

Давайте попробуем разморозить веса BERT и тренировать их. Это требует очень малой скорости обучения, а также более тщательной стратегии обучения с **разминкой**, с использованием оптимизатора **AdamW**. Мы будем использовать пакет `tf-models-official` для создания оптимизатора:

In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

Как видите, обучение идет довольно медленно - но вы можете поэкспериментировать и обучить модель в течение нескольких эпох (5-10) и посмотреть, сможете ли вы получить лучший результат по сравнению с подходами, которые мы использовали раньше.

## Библиотека Huggingface Transformers

Другим очень распространенным (и немного более простым) способом использования моделей Transformer является [пакет HuggingFace](https://github.com/huggingface/), который предоставляет своего рода *строительные блоки* для различных задач NLP. Он доступен как для Tensorflow, так и для PyTorch, еще одного очень популярного фреймворка нейронных сетей. 

> **Примечание**: Ниже мы будем повторять те же шаги обучения модели BERT с использованием другой библиотеки и существенно большей модели. Таким образом, процесс включает в себя довольно длительное обучение, поэтому вы можете просто просмотреть код.

Let's see how our problem can be solved using [Huggingface Transformers](http://huggingface.co).

Давайте посмотрим, как наша проблема может быть решена с помощью [Трансформеры Huggingface] (http://huggingface.co).

First thing we need to do is to chose the model that we will be using. In addition to some built-in models, Huggingface contains an [online model repository](https://huggingface.co/models), where you can find a lot more pre-trained models by the community. All of those models can be loaded and used just by providing a model name. All required binary files for the model would automatically be downloaded.


Первое, что нам нужно сделать, это выбрать модель, которую мы будем использовать. В дополнение к некоторым встроенным моделям, Huggingface содержит [онлайн-репозиторий моделей] (https://huggingface.co/models), где вы можете найти гораздо больше предварительно обученных моделей сообществом. Все эти модели можно загрузить и использовать, просто указав имя модели. Все необходимые двоичные файлы для модели будут загружены автоматически.

At certain times you would need to load your own models, in which case you can specify the directory that contains all relevant files, including parameters for tokenizer, `config.json` file with model parameters, binary weights, etc.


В определенное время вам нужно будет загрузить свои собственные модели, и в этом случае вы можете указать каталог, который содержит все соответствующие файлы, включая параметры для токенизатора, файл 'config.json' с параметрами модели, двоичные веса и т. Д.

From model name, we can instantiate both the model and the tokenizer. Let's start with a tokenizer:

Из имени модели мы можем создать экземпляр как модели, так и токенизатора. Начнем с токенизатора:

In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

The `tokenizer` object contains the `encode` function that can be directly used to encode text:

Объект 'tokenizer' содержит функцию 'encode', которую можно напрямую использовать для кодирования текста:

In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

We can also use tokenizer to encode a sequence is a way suitable for passing to the model, i.e. including `token_ids`, `input_mask` fields, etc. We can also specify that we want Tensorflow tensors by providing `return_tensors='tf'` argument:

Мы также можем использовать токенизатор для кодирования последовательности — способа, подходящего для передачи в модель, т.е. включения полей «token_ids», «input_mask» и т.д. Мы также можем указать, что нам нужны тензоры Tensorflow, предоставив аргумент 'return_tensors='tf'':

In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

In our case, we will be using pre-trained BERT model called `bert-base-uncased`. *Uncased* indicates that the model in case-insensitive. 


В нашем случае мы будем использовать предварительно обученную модель BERT под названием 'bert-base-uncased'. *Без регистра* указывает, что модель не чувствительна к регистру. 

When training the model, we need to provide tokenized sequence as input, and thus we will design data processing pipeline. Since `tokenizer.encode` is a Python function, we will use the same approach as in the last unit with calling it using `py_function`:

При обучении модели нам нужно предоставить токенизированную последовательность в качестве входных данных, и таким образом мы спроектируем конвейер обработки данных. Поскольку 'tokenizer.encode' является функцией Python, мы будем использовать тот же подход, что и в последнем блоке, с вызовом его с помощью 'py_function':

In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

Now we can load the actual model using `BertForSequenceClassfication` package. This ensures that our model already has a required architecture for classification, including final classifier. You will see warning message stating that weights of the final classifier are not initialized, and model would require pre-training - that is perfectly okay, because it is exactly what we are about to do!

Теперь мы можем загрузить фактическую модель с помощью пакета 'BertForSequenceClassfication'. Это гарантирует, что наша модель уже имеет необходимую архитектуру для классификации, включая окончательный классификатор. Вы увидите предупреждающее сообщение о том, что веса конечного классификатора не инициализируются, а модель потребует предварительной подготовки - это совершенно нормально, потому что это именно то, что мы собираемся сделать!

In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


As you can see from `summary()`, the model contains almost 110 million parameters! Presumably, if we want simple classification task on relatively small dataset, we do not want to train the BERT base layer:

Как видно из 'summary()', модель содержит почти 110 миллионов параметров! Предположительно, если нам нужна простая задача классификации относительно небольшого набора данных, мы не хотим обучать базовый слой BERT:

In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


Now we are ready to begin training!


Теперь мы готовы приступить к тренировкам!

> **Note**: Training full-scale BERT model can be very time consuming! Thus we will only train it for the first 32 batches. This is just to show how model training is set up. If you are interested to try full-scale training - just remove `steps_per_epoch` and `validation_steps` parameters, and prepare to wait!

> **Примечание**: Обучение полномасштабной модели BERT может занять очень много времени! Таким образом, мы будем обучать его только для первых 32 партий. Это просто для того, чтобы показать, как настраивается обучение моделей. Если вам интересно попробовать полномасштабное обучение - просто уберите параметры «steps_per_epoch» и «validation_steps» и приготовьтесь ждать!

In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


If you increase the number of iterations and wait long enough, and train for several epochs, you can expect that BERT classification gives us the best accuracy! That is because BERT already understands quite well the structure of the language, and we only need to fine-tune final classifier. However, because BERT is a large model, the whole training process takes a long time, and requires serious computational power! (GPU, and preferably more than one).


Если увеличить количество итераций и ждать достаточно долго, и тренироваться в течение нескольких эпох, то можно ожидать, что классификация BERT даст нам наилучшую точность! Это связано с тем, что BERT уже довольно хорошо понимает структуру языка, и нам нужно только точно настроить окончательный классификатор. Однако, поскольку BERT является большой моделью, весь процесс обучения занимает много времени и требует серьезных вычислительных мощностей! (GPU, и желательно более одного).

> **Note:** In our example, we have been using one of the smallest pre-trained BERT models. There are larger models that are likely to yield better results.

> **Примечание:** В нашем примере мы использовали одну из самых маленьких предварительно обученных моделей BERT. Существуют более крупные модели, которые, вероятно, дадут лучшие результаты.

## Takeaway


## Еда на вынос

In this unit, we have seen very recent model architectures based on **transformers**. We have applied them for our text classification task, but similarly, BERT models can be used for entity extraction, question answering, and other NLP tasks.


В этом устройстве мы видели совсем последние модели архитектур на основе **трансформаторов**. Мы применили их для нашей задачи классификации текста, но аналогичным образом модели BERT можно использовать для извлечения сущностей, ответов на вопросы и других задач НЛП.

Transformer models represent current state-of-the-art in NLP, and in most of the cases it should be the first solution you start experimenting with when implementing custom NLP solutions. However, understanding basic underlying principles of recurrent neural networks discussed in this module is extremely important if you want to build advanced neural models.

Модели трансформаторов представляют собой современное состояние НЛП, и в большинстве случаев это должно быть первое решение, с которым вы начинаете экспериментировать при реализации пользовательских решений НЛП. Однако понимание основных основополагающих принципов рекуррентных нейронных сетей, обсуждаемых в этом модуле, чрезвычайно важно, если вы хотите построить продвинутые нейронные модели.